In [ ]:
import wget

url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
output_file = "./cats_and_dogs_filtered.zip"

wget.download(url, out=output_file)

'./cats_and_dogs_filtered.zip'

In [3]:
train_folder = './data/cats_and_dogs/train'
valid_folder = './data/cats_and_dogs/validation'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1/255.0,
                                     rotation_range = 20,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)

valid_generator = ImageDataGenerator(rescale = 1/255.0)

generated_train_data = train_generator.flow_from_directory(train_folder,
                                                           batch_size = 64,
                                                           class_mode = 'binary',
                                                           target_size = (150, 150))
generated_valid_data = valid_generator.flow_from_directory(valid_folder,
                                                           batch_size = 64,
                                                           class_mode = 'binary',
                                                           target_size = (150, 150))

Found 2010 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [5]:
pre_trained_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (150, 150, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in pre_trained_model.layers:
    layer.trainable = False
pre_trained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
model = Sequential()
model.add(pre_trained_model)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 23105345 (88.14 MB)
Trainable params: 8390657 (32.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [8]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(generated_train_data, validation_data = generated_valid_data, epochs = 10, verbose = 1)

Epoch 1/10
32/32 [==============================] - 460s 14s/step - loss: 0.7429 - accuracy: 0.7438 - val_loss: 0.3188 - val_accuracy: 0.8630
Epoch 2/10
32/32 [==============================] - 36s 1s/step - loss: 0.3688 - accuracy: 0.8249 - val_loss: 0.2806 - val_accuracy: 0.8720
Epoch 3/10
32/32 [==============================] - 32s 989ms/step - loss: 0.3261 - accuracy: 0.8602 - val_loss: 0.3234 - val_accuracy: 0.8630
Epoch 4/10
32/32 [==============================] - 32s 989ms/step - loss: 0.3269 - accuracy: 0.8502 - val_loss: 0.2620 - val_accuracy: 0.8920
Epoch 5/10
32/32 [==============================] - 32s 1s/step - loss: 0.3266 - accuracy: 0.8592 - val_loss: 0.2501 - val_accuracy: 0.8880
Epoch 6/10
32/32 [==============================] - 34s 1s/step - loss: 0.2827 - accuracy: 0.8851 - val_loss: 0.2632 - val_accuracy: 0.8940
Epoch 7/10
32/32 [==============================] - 31s 979ms/step - loss: 0.3028 - accuracy: 0.8667 - val_loss: 0.2947 - val_accuracy: 0.8740
Epoch 8/1